In [1]:
# Dependencies
import pandas as pd
import csv
import numpy as np

import re

from config import password
from sqlalchemy import create_engine

# CSV path
goodreads_file = "Resources/goodreads.csv"

In [2]:
# Read CSV
goodreads_df = pd.read_csv(goodreads_file)
goodreads_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count,extra
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613,NaN
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390,NaN
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272,NaN
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964,NaN


In [3]:
# Create new dataframe with select columns
#goodreads_df.drop(columns=['isbn', 'isbn13', '# num_pages', 'extra'], inplace=True)
new_goodreads_df = goodreads_df[['bookID', 'title', 'authors', 'average_rating','language_code', 'ratings_count',
                                'text_reviews_count']].copy()

# Rename columns
new_goodreads_df.columns = ['book_id', 'title', 'authors', 'avg_rating', 
                        'language_code', 'ratings_count', 'text_reviews_count']

# Set index to book_id
new_goodreads_df.set_index('book_id', inplace=True)

new_goodreads_df.head()

,title,authors,avg_rating,language_code,ratings_count,text_reviews_count
book_id,,,,,,
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,eng,1944099,26249
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,eng,1996446,27613
3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,eng,5629932,70390
4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,eng,6267,272
5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,eng,2149872,33964


In [4]:
# Condition to check if all rows under avg_rating column are numeric
condition = [bool(re.match("[0-9]{1}\.[0-9]{2}",x)) for x in new_goodreads_df["avg_rating"].to_list()]

In [5]:
# Apply the condition to dataframe
goodreads_transformed = new_goodreads_df[condition]
goodreads_transformed.head()

,title,authors,avg_rating,language_code,ratings_count,text_reviews_count
book_id,,,,,,
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,eng,1944099,26249
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,eng,1996446,27613
3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,eng,5629932,70390
4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,eng,6267,272
5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,eng,2149872,33964


In [6]:
# Create Postgres engine
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/books_db')

In [7]:
# Read tables
engine.table_names()

['goodreads']

In [10]:
# Insert data into postgres table
goodreads_transformed.to_sql(name='goodreads', con=engine, if_exists='replace', index=True)

In [11]:
# Confirm data stored in the database
pd.read_sql_query('select * from goodreads', con=engine).head()

,book_id,title,authors,avg_rating,language_code,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,eng,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,eng,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,eng,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,eng,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,eng,2149872,33964
